In [1]:
import pandas as pd
import pgeocode
import json
import numpy as np

# Test Data

In [2]:
test_data = pd.read_csv('test_data_question.csv')
test_data

,CUSTOMER_ID,STORE_NUMBER,ORDER_ID,ORDER_CHANNEL_NAME,ORDER_SUBCHANNEL_NAME,ORDER_OCCASION_NAME,CUSTOMER_TYPE,item1,item2,item3
0,997177535,4915,9351345556,Digital,WWT,ToGo,Guest,Chicken Sub Combo,Ranch Dip - Regular,10 pc Spicy Wings Combo
1,345593831,949,3595377080,Digital,WWT,ToGo,Registered,Regular Buffalo Fries,10 pc Spicy Wings,3 pc Crispy Strips Combo
2,160955031,2249,4071757785,Digital,WWT,ToGo,Guest,Large Buffalo Fries,10 pc Spicy Wings,Ranch Dip - Regular
3,890671991,4154,3931766769,Digital,WWT,ToGo,Guest,6 pc Grilled Wings Combo,20 pc Grilled Wings,Fried Corn - Large
4,73989021,4094,3739700809,Digital,WWT,ToGo,Registered,Regular Buffalo Fries,20 pc Grilled Wings,Ranch Dip - Large
...,...,...,...,...,...,...,...,...,...,...
995,199551661,4915,5790578362,Digital,WWT,Delivery,Registered,Chicken Sub Combo,10 pc Grilled Wings,Chicken Sub
996,324907688,1161,371107248,Digital,WWT,ToGo,Guest,10 pc Grilled Wings Combo,Veggie Sticks Spicy,8 pc Grilled Wings Combo
997,688871063,949,7519184133,Digital,WWT,ToGo,Registered,Fried Corn - Regular,Flavor Platter,Ranch Dip - Regular
998,529892223,4750,8800902863,Digital,WWT,Delivery,Registered,Regular Buffalo Fries,Ranch Dip - Regular,Legendary Feast Bundle


In [3]:
test_data.nunique()

CUSTOMER_ID              1000
STORE_NUMBER               38
ORDER_ID                 1000
ORDER_CHANNEL_NAME          1
ORDER_SUBCHANNEL_NAME       1
ORDER_OCCASION_NAME         2
CUSTOMER_TYPE               3
item1                      75
item2                      79
item3                      79
dtype: int64

In [4]:
test_data['CUSTOMER_TYPE'].value_counts()

CUSTOMER_TYPE
Registered    753
Guest         246
eClub           1
Name: count, dtype: int64

In [5]:
test_data['ORDER_CHANNEL_NAME'].value_counts()

ORDER_CHANNEL_NAME
Digital    1000
Name: count, dtype: int64

In [6]:
test_data['ORDER_SUBCHANNEL_NAME'].value_counts()

ORDER_SUBCHANNEL_NAME
WWT    1000
Name: count, dtype: int64

In [7]:
test_data['ORDER_OCCASION_NAME'].value_counts()

ORDER_OCCASION_NAME
ToGo        853
Delivery    147
Name: count, dtype: int64

In [8]:
test_data['item1'].unique()

array(['Chicken Sub Combo', 'Regular Buffalo Fries',
       'Large Buffalo Fries', '6 pc Grilled Wings Combo',
       '3 pc Crispy Strips Combo', '10 pc Spicy Wings',
       'Ranch Dip - Regular', 'Fried Corn - Regular',
       '20 pc Grilled Wings', '8 pc Spicy Wings Combo',
       '15 pc Spicy Wings Combo', '2 pc Crispy Strips',
       '30 pc Family Grilled Wings', 'Honey Mustard Dip - Regular',
       '3 Strips Lunch', 'Add 5 Grilled Wings', '10 pc Grilled Wings',
       'Chicken Sub', '20pc Spicy Feast Deal', 'Voodoo Fries - Regular',
       '4 pc Crispy Strips', 'Fried Corn - Large',
       '15 pc Mixed Wings Combo', '6 pc Spicy Wings Combo',
       'Cheese Fries - Large', 'Ranch Dip - Large', '32 Oz Soda',
       '10 pc Grilled Wings Combo', 'Veggie Sticks Spicy',
       '10 pc Mixed Wings Combo', '6 pc Strips Meal for 2', '20 Oz Soda',
       'Carrot Sticks', '20 pc Mixed Wings', 'Large Veggie Sticks Spicy',
       '5 pc Crispy Strips Combo', '15 pc Grilled Wings Combo',
       

In [9]:
test_data['item2'].unique()

array(['Ranch Dip - Regular', '10 pc Spicy Wings', '20 pc Grilled Wings',
       'Chicken Sub Combo', 'Add 5 Spicy Wings', 'Chicken Sub',
       '8 pc Grilled Wings Combo', '5 pc Crispy Strips Combo',
       '6 pc Grilled Wings Combo', '6 pc Spicy Wings Combo',
       '2 pc Crispy Strips', 'Large Buffalo Fries', '10 pc Grilled Wings',
       'Ranch Dip - Large', '20 Oz Soda', '6 pc Mixed Wings Combo',
       '15 pc Grilled Wings', '8 pc Spicy Wings Combo',
       'Triple Chocolate Cake', '50 pc Grilled Wings',
       '10 pc Mixed Wings', '20 pc Mixed Wings',
       '3 pc Crispy Strips Combo', '20pc Spicy Feast Deal',
       'Add 5 Grilled Wings', 'Blue Cheese Dip - Regular', '32 Oz Soda',
       '15 pc Crispy Strips', 'Regular Buffalo Fries',
       '30 pc Spicy Wings', '10 pc Grilled Wings Combo',
       '30 pc Family Grilled Wings', '4 pc Crispy Strips',
       'Voodoo Fries - Regular', 'Fried Corn - Large',
       '50 pc Mixed Wings', 'Cheese Fries - Large', '7 pc Crispy Strips',
  

In [10]:
test_data['item3'].unique()

array(['10 pc Spicy Wings Combo', '3 pc Crispy Strips Combo',
       'Ranch Dip - Regular', 'Fried Corn - Large', 'Ranch Dip - Large',
       'Chicken Sub', 'Veggie Sticks Spicy', '8 pc Spicy Wings Combo',
       'Regular Buffalo Fries', 'Chicken Sub Combo',
       'Voodoo Fries - Regular', '10 pc Spicy Wings',
       '5 pc Crispy Strips Combo', '20pc Spicy Feast Deal',
       '10 pc Grilled Wings Combo', '10 pc Grilled Wings',
       'Cheese Fries - Large', '2 pc Crispy Strips',
       '8 pc Grilled Wings Combo', 'Fried Corn - Regular',
       '7 pc Crispy Strips', '20 Oz Soda', '15 pc Grilled Wings',
       'Voodoo Fries - Large', 'Large Buffalo Fries', 'Carrot Sticks',
       '6 pc Grilled Wings Combo', 'Flavor Platter', 'Veggie Sticks',
       '15 pc Spicy Wings', '40 pc Family Mixed Wings',
       'Large Veggie Sticks Spicy', 'Cheese Dip - Medium',
       '4 pc Crispy Strips', '20 pc Grilled Wings',
       'Cheese Fries - Regular', 'Triple Chocolate Cake',
       '15 pc Spicy Wing

# 1. Customer Data

In [17]:
customer_data = pd.read_csv('customer_data.csv')
customer_data

,CUSTOMER_ID,CUSTOMER_TYPE
0,362204699,Registered
1,269612955,Registered
2,585330633,Guest
3,950661333,Registered
4,434985772,Guest
...,...,...
563341,816761901,Guest
563342,142930310,Registered
563343,443106950,Registered
563344,614873531,Registered


In [18]:
customer_data.shape

(563346, 2)

In [19]:
customer_data.nunique()

CUSTOMER_ID      563346
CUSTOMER_TYPE         5
dtype: int64

In [20]:
customer_data.isna().sum()

CUSTOMER_ID       0
CUSTOMER_TYPE    27
dtype: int64

In [21]:
customer_data.duplicated().any()

False

In [22]:
customer_data['CUSTOMER_TYPE'].value_counts()

CUSTOMER_TYPE
Registered         406653
Guest              155132
eClub                1184
Deleted Account       348
Online                  2
Name: count, dtype: int64

In [23]:
customer_data[customer_data['CUSTOMER_TYPE'].isna()]

,CUSTOMER_ID,CUSTOMER_TYPE
4435,495942367,NaN
49899,486166137,NaN
70611,42901498,NaN
75051,953490107,NaN
78020,55242064,NaN
85913,143933193,NaN
99768,411458861,NaN
100265,27866166,NaN
124326,339524761,NaN
135377,236397925,NaN


### Replacing NA values with 'Unknown' Customer Type

In [24]:
customer_data['CUSTOMER_TYPE'].fillna('Unknown', inplace=True)

In [25]:
customer_data.isna().sum()

CUSTOMER_ID      0
CUSTOMER_TYPE    0
dtype: int64

In [26]:
customer_data.nunique()

CUSTOMER_ID      563346
CUSTOMER_TYPE         6
dtype: int64

In [27]:
customer_data['CUSTOMER_TYPE'].value_counts()

CUSTOMER_TYPE
Registered         406653
Guest              155132
eClub                1184
Deleted Account       348
Unknown                27
Online                  2
Name: count, dtype: int64

In [28]:
customer_data

,CUSTOMER_ID,CUSTOMER_TYPE
0,362204699,Registered
1,269612955,Registered
2,585330633,Guest
3,950661333,Registered
4,434985772,Guest
...,...,...
563341,816761901,Guest
563342,142930310,Registered
563343,443106950,Registered
563344,614873531,Registered


# 2. Store Data

In [29]:
store_data = pd.read_csv('store_data.csv')
store_data

,STORE_NUMBER,CITY,STATE,POSTAL_CODE
0,2156,GRAPEVINE,TX,76051
1,1419,HUNTERSVILLE,NC,28078
2,2249,NaN,NaN,32792
3,2513,LAS VEGAS,NV,89129
4,1754,ARDMORE,OK,73401
5,949,NaN,NaN,93033
6,820,NaN,NaN,37203
7,1161,NaN,NaN,78723-2429
8,2517,RUTHERFORD,CA,94573
9,4391,OMAHA,NE,68107


In [30]:
store_data['POSTAL_CODE']

0          76051
1          28078
2          32792
3          89129
4          73401
5          93033
6          37203
7     78723-2429
8          94573
9          68107
10         75063
11         28208
12         76002
13         33511
14    75231-0242
15         75654
16         48390
17         77025
18         92308
19         78245
20           NaN
21         48044
22           NaN
23         95301
24         78261
25         33125
26         77011
27         78543
28         80013
29         96789
30         75218
31         07036
32         60804
33         79905
34         85255
35         77047
36         28278
37         80120
Name: POSTAL_CODE, dtype: object

In [31]:
store_data['POSTAL_CODE'] = store_data['POSTAL_CODE'].astype(str).str[:5]

In [32]:
store_data

,STORE_NUMBER,CITY,STATE,POSTAL_CODE
0,2156,GRAPEVINE,TX,76051
1,1419,HUNTERSVILLE,NC,28078
2,2249,NaN,NaN,32792
3,2513,LAS VEGAS,NV,89129
4,1754,ARDMORE,OK,73401
5,949,NaN,NaN,93033
6,820,NaN,NaN,37203
7,1161,NaN,NaN,78723
8,2517,RUTHERFORD,CA,94573
9,4391,OMAHA,NE,68107


### Fetching data for Unknown Locations using Postal Codes, and vice-versa

In [33]:
# Initialize for the US
nomi = pgeocode.Nominatim('us')

# Example for a single ZIP
nomi.query_postal_code("90210")  # returns city, state_name, etc.

# For your dataframe:
store_data['city_filled'] = store_data['POSTAL_CODE'].apply(lambda x: nomi.query_postal_code(str(x)).place_name)
store_data['state_filled'] = store_data['POSTAL_CODE'].apply(lambda x: nomi.query_postal_code(str(x)).state_name)

In [34]:
store_data

,STORE_NUMBER,CITY,STATE,POSTAL_CODE,city_filled,state_filled
0,2156,GRAPEVINE,TX,76051,Grapevine,Texas
1,1419,HUNTERSVILLE,NC,28078,Huntersville,North Carolina
2,2249,NaN,NaN,32792,Winter Park,Florida
3,2513,LAS VEGAS,NV,89129,Las Vegas,Nevada
4,1754,ARDMORE,OK,73401,Ardmore,Oklahoma
5,949,NaN,NaN,93033,Oxnard,California
6,820,NaN,NaN,37203,Nashville,Tennessee
7,1161,NaN,NaN,78723,Austin,Texas
8,2517,RUTHERFORD,CA,94573,Rutherford,California
9,4391,OMAHA,NE,68107,Omaha,Nebraska


In [35]:
store_data.isna().sum()

STORE_NUMBER     0
CITY            13
STATE           14
POSTAL_CODE      0
city_filled      2
state_filled     2
dtype: int64

In [36]:
store_data.dtypes

STORE_NUMBER     int64
CITY            object
STATE           object
POSTAL_CODE     object
city_filled     object
state_filled    object
dtype: object

In [37]:
store_data[store_data['CITY'] == 'LAVEEN']

,STORE_NUMBER,CITY,STATE,POSTAL_CODE,city_filled,state_filled
20,1731,LAVEEN,NaN,nan,NaN,NaN


In [38]:
store_data.loc[store_data['CITY'] == 'LAVEEN', 'POSTAL_CODE'] = '85339'
store_data.loc[store_data['CITY'] == 'LAVEEN', 'STATE'] = 'AZ'
store_data.loc[store_data['CITY'] == 'LAVEEN', 'city_filled'] = 'Laveen'
store_data.loc[store_data['CITY'] == 'LAVEEN', 'state_filled'] = 'Arizona'

In [39]:
store_data

,STORE_NUMBER,CITY,STATE,POSTAL_CODE,city_filled,state_filled
0,2156,GRAPEVINE,TX,76051,Grapevine,Texas
1,1419,HUNTERSVILLE,NC,28078,Huntersville,North Carolina
2,2249,NaN,NaN,32792,Winter Park,Florida
3,2513,LAS VEGAS,NV,89129,Las Vegas,Nevada
4,1754,ARDMORE,OK,73401,Ardmore,Oklahoma
5,949,NaN,NaN,93033,Oxnard,California
6,820,NaN,NaN,37203,Nashville,Tennessee
7,1161,NaN,NaN,78723,Austin,Texas
8,2517,RUTHERFORD,CA,94573,Rutherford,California
9,4391,OMAHA,NE,68107,Omaha,Nebraska


In [40]:
store_data.isna().sum()

STORE_NUMBER     0
CITY            13
STATE           13
POSTAL_CODE      0
city_filled      1
state_filled     1
dtype: int64

In [41]:
store_data['CITY'] = store_data['city_filled'].str.upper()

In [42]:
store_data

,STORE_NUMBER,CITY,STATE,POSTAL_CODE,city_filled,state_filled
0,2156,GRAPEVINE,TX,76051,Grapevine,Texas
1,1419,HUNTERSVILLE,NC,28078,Huntersville,North Carolina
2,2249,WINTER PARK,NaN,32792,Winter Park,Florida
3,2513,LAS VEGAS,NV,89129,Las Vegas,Nevada
4,1754,ARDMORE,OK,73401,Ardmore,Oklahoma
5,949,OXNARD,NaN,93033,Oxnard,California
6,820,NASHVILLE,NaN,37203,Nashville,Tennessee
7,1161,AUSTIN,NaN,78723,Austin,Texas
8,2517,RUTHERFORD,CA,94573,Rutherford,California
9,4391,OMAHA,NE,68107,Omaha,Nebraska


In [43]:
store_data.isna().sum()

STORE_NUMBER     0
CITY             1
STATE           13
POSTAL_CODE      0
city_filled      1
state_filled     1
dtype: int64

In [44]:
store_data

,STORE_NUMBER,CITY,STATE,POSTAL_CODE,city_filled,state_filled
0,2156,GRAPEVINE,TX,76051,Grapevine,Texas
1,1419,HUNTERSVILLE,NC,28078,Huntersville,North Carolina
2,2249,WINTER PARK,NaN,32792,Winter Park,Florida
3,2513,LAS VEGAS,NV,89129,Las Vegas,Nevada
4,1754,ARDMORE,OK,73401,Ardmore,Oklahoma
5,949,OXNARD,NaN,93033,Oxnard,California
6,820,NASHVILLE,NaN,37203,Nashville,Tennessee
7,1161,AUSTIN,NaN,78723,Austin,Texas
8,2517,RUTHERFORD,CA,94573,Rutherford,California
9,4391,OMAHA,NE,68107,Omaha,Nebraska


In [45]:
store_data['state_filled']

0              Texas
1     North Carolina
2            Florida
3             Nevada
4           Oklahoma
5         California
6          Tennessee
7              Texas
8         California
9           Nebraska
10             Texas
11    North Carolina
12             Texas
13           Florida
14             Texas
15             Texas
16          Michigan
17             Texas
18        California
19             Texas
20           Arizona
21          Michigan
22               NaN
23        California
24             Texas
25           Florida
26             Texas
27             Texas
28          Colorado
29            Hawaii
30             Texas
31        New Jersey
32          Illinois
33             Texas
34           Arizona
35             Texas
36    North Carolina
37          Colorado
Name: state_filled, dtype: object

### US States and their Abbreviations

In [46]:
# Dictionary: Full state name → Abbreviation
state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT',
    'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI',
    'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME',
    'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI',
    'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
    'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH',
    'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA',
    'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY'
}

# Map full name to abbreviation
store_data['state_abbrev'] = store_data['state_filled'].map(state_abbrev)

In [47]:
store_data

,STORE_NUMBER,CITY,STATE,POSTAL_CODE,city_filled,state_filled,state_abbrev
0,2156,GRAPEVINE,TX,76051,Grapevine,Texas,TX
1,1419,HUNTERSVILLE,NC,28078,Huntersville,North Carolina,NC
2,2249,WINTER PARK,NaN,32792,Winter Park,Florida,FL
3,2513,LAS VEGAS,NV,89129,Las Vegas,Nevada,NV
4,1754,ARDMORE,OK,73401,Ardmore,Oklahoma,OK
5,949,OXNARD,NaN,93033,Oxnard,California,CA
6,820,NASHVILLE,NaN,37203,Nashville,Tennessee,TN
7,1161,AUSTIN,NaN,78723,Austin,Texas,TX
8,2517,RUTHERFORD,CA,94573,Rutherford,California,CA
9,4391,OMAHA,NE,68107,Omaha,Nebraska,NE


In [48]:
store_data.isna().sum()

STORE_NUMBER     0
CITY             1
STATE           13
POSTAL_CODE      0
city_filled      1
state_filled     1
state_abbrev     1
dtype: int64

In [49]:
store_data['STATE'] = store_data['state_abbrev']

In [50]:
store_data.isna().sum()

STORE_NUMBER    0
CITY            1
STATE           1
POSTAL_CODE     0
city_filled     1
state_filled    1
state_abbrev    1
dtype: int64

In [51]:
store_data

,STORE_NUMBER,CITY,STATE,POSTAL_CODE,city_filled,state_filled,state_abbrev
0,2156,GRAPEVINE,TX,76051,Grapevine,Texas,TX
1,1419,HUNTERSVILLE,NC,28078,Huntersville,North Carolina,NC
2,2249,WINTER PARK,FL,32792,Winter Park,Florida,FL
3,2513,LAS VEGAS,NV,89129,Las Vegas,Nevada,NV
4,1754,ARDMORE,OK,73401,Ardmore,Oklahoma,OK
5,949,OXNARD,CA,93033,Oxnard,California,CA
6,820,NASHVILLE,TN,37203,Nashville,Tennessee,TN
7,1161,AUSTIN,TX,78723,Austin,Texas,TX
8,2517,RUTHERFORD,CA,94573,Rutherford,California,CA
9,4391,OMAHA,NE,68107,Omaha,Nebraska,NE


In [52]:
store_data[store_data['STORE_NUMBER'] == 4750]

,STORE_NUMBER,CITY,STATE,POSTAL_CODE,city_filled,state_filled,state_abbrev
22,4750,NaN,NaN,nan,NaN,NaN,NaN


In [53]:
store_data.loc[store_data['STORE_NUMBER'] == 4750, ['CITY', 'STATE', 'POSTAL_CODE', 'city_filled', 'state_filled', 'state_abbrev']] = 'Unknown'

In [54]:
store_data

,STORE_NUMBER,CITY,STATE,POSTAL_CODE,city_filled,state_filled,state_abbrev
0,2156,GRAPEVINE,TX,76051,Grapevine,Texas,TX
1,1419,HUNTERSVILLE,NC,28078,Huntersville,North Carolina,NC
2,2249,WINTER PARK,FL,32792,Winter Park,Florida,FL
3,2513,LAS VEGAS,NV,89129,Las Vegas,Nevada,NV
4,1754,ARDMORE,OK,73401,Ardmore,Oklahoma,OK
5,949,OXNARD,CA,93033,Oxnard,California,CA
6,820,NASHVILLE,TN,37203,Nashville,Tennessee,TN
7,1161,AUSTIN,TX,78723,Austin,Texas,TX
8,2517,RUTHERFORD,CA,94573,Rutherford,California,CA
9,4391,OMAHA,NE,68107,Omaha,Nebraska,NE


In [55]:
store_data.isna().sum()

STORE_NUMBER    0
CITY            0
STATE           0
POSTAL_CODE     0
city_filled     0
state_filled    0
state_abbrev    0
dtype: int64

In [56]:
store_data['STORE_NUMBER'].nunique()

38

In [57]:
store_data

,STORE_NUMBER,CITY,STATE,POSTAL_CODE,city_filled,state_filled,state_abbrev
0,2156,GRAPEVINE,TX,76051,Grapevine,Texas,TX
1,1419,HUNTERSVILLE,NC,28078,Huntersville,North Carolina,NC
2,2249,WINTER PARK,FL,32792,Winter Park,Florida,FL
3,2513,LAS VEGAS,NV,89129,Las Vegas,Nevada,NV
4,1754,ARDMORE,OK,73401,Ardmore,Oklahoma,OK
5,949,OXNARD,CA,93033,Oxnard,California,CA
6,820,NASHVILLE,TN,37203,Nashville,Tennessee,TN
7,1161,AUSTIN,TX,78723,Austin,Texas,TX
8,2517,RUTHERFORD,CA,94573,Rutherford,California,CA
9,4391,OMAHA,NE,68107,Omaha,Nebraska,NE


In [123]:
store_data.nunique()

STORE_NUMBER    38
CITY            33
STATE           15
POSTAL_CODE     38
city_filled     33
state_filled    15
state_abbrev    15
dtype: int64

# 3. Orders Data

In [58]:
order_data = pd.read_csv('order_data.csv')
order_data

,CUSTOMER_ID,STORE_NUMBER,ORDER_CREATED_DATE,ORDER_ID,ORDERS,ORDER_CHANNEL_NAME,ORDER_SUBCHANNEL_NAME,ORDER_OCCASION_NAME
0,362204699,2156,2024-07-24,7247194287,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo
1,269612955,1419,2025-02-15,791214421,"{""orders"": [{""item_details"": [{""item_name"": ""R...",Digital,WWT,ToGo
2,585330633,2249,2025-02-15,7575285208,"{""orders"": [{""item_details"": [{""item_name"": ""2...",Digital,WWT,ToGo
3,950661333,2513,2024-03-29,4253875716,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo
4,434985772,1754,2024-04-08,7150407872,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo
...,...,...,...,...,...,...,...,...
1414405,125634365,4595,2025-02-02,762304305,"{""orders"": [{""item_details"": [{""item_name"": ""1...",Digital,WWT,ToGo
1414406,125634365,1419,2025-01-22,9371418183,"{""orders"": [{""item_details"": [{""item_name"": ""C...",Digital,WWT,ToGo
1414407,125634365,4595,2025-02-13,8082200700,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo
1414408,125634365,1419,2024-11-30,8865782702,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo


In [59]:
order_data.isna().sum()

CUSTOMER_ID              0
STORE_NUMBER             0
ORDER_CREATED_DATE       0
ORDER_ID                 0
ORDERS                   0
ORDER_CHANNEL_NAME       0
ORDER_SUBCHANNEL_NAME    0
ORDER_OCCASION_NAME      0
dtype: int64

In [60]:
order_data.duplicated().any()

False

In [61]:
order_data.nunique()

CUSTOMER_ID               563346
STORE_NUMBER                  38
ORDER_CREATED_DATE           446
ORDER_ID                 1414410
ORDERS                    735527
ORDER_CHANNEL_NAME             1
ORDER_SUBCHANNEL_NAME          2
ORDER_OCCASION_NAME            2
dtype: int64

In [74]:
order_data.groupby(by='CUSTOMER_ID')['ORDER_ID'].nunique().sort_values(ascending=False).head(10)

CUSTOMER_ID
522408835    2802
575231786     163
300120733     154
807861401     108
377803146     107
593414163      97
305682849      96
22394267       93
757801678      93
418779566      90
Name: ORDER_ID, dtype: int64

In [66]:
order_data['ORDER_CHANNEL_NAME'].value_counts()

ORDER_CHANNEL_NAME
Digital    1414410
Name: count, dtype: int64

In [67]:
order_data['ORDER_SUBCHANNEL_NAME'].value_counts()

ORDER_SUBCHANNEL_NAME
WWT                     1411089
Grub Hub Marketplace       3321
Name: count, dtype: int64

In [68]:
order_data['ORDER_OCCASION_NAME'].value_counts()

ORDER_OCCASION_NAME
ToGo        1214921
Delivery     199489
Name: count, dtype: int64

In [69]:
order_data['ORDERS']

0          {"orders": [{"item_details": [{"item_name": "O...
1          {"orders": [{"item_details": [{"item_name": "R...
2          {"orders": [{"item_details": [{"item_name": "2...
3          {"orders": [{"item_details": [{"item_name": "O...
4          {"orders": [{"item_details": [{"item_name": "O...
                                 ...                        
1414405    {"orders": [{"item_details": [{"item_name": "1...
1414406    {"orders": [{"item_details": [{"item_name": "C...
1414407    {"orders": [{"item_details": [{"item_name": "O...
1414408    {"orders": [{"item_details": [{"item_name": "O...
1414409    {"orders": [{"item_details": [{"item_name": "R...
Name: ORDERS, Length: 1414410, dtype: object

In [70]:
# View full JSON of row 0
print(json.dumps(json.loads(order_data.loc[0, 'ORDERS']), indent=2))

{
  "orders": [
    {
      "item_details": [
        {
          "item_name": "Order Memo Not Paid",
          "item_price": 0,
          "item_quantity": 1
        },
        {
          "item_name": "10 pc Grilled Wings Combo",
          "item_price": 15.29,
          "item_quantity": 1
        },
        {
          "item_name": "8 pc Grilled Wings Combo",
          "item_price": 13.29,
          "item_quantity": 1
        },
        {
          "item_name": "8 pc Spicy Wings Combo",
          "item_price": 10.99,
          "item_quantity": 1
        }
      ]
    }
  ]
}


In [71]:
order_data.columns

Index(['CUSTOMER_ID', 'STORE_NUMBER', 'ORDER_CREATED_DATE', 'ORDER_ID',
       'ORDERS', 'ORDER_CHANNEL_NAME', 'ORDER_SUBCHANNEL_NAME',
       'ORDER_OCCASION_NAME'],
      dtype='object')

### Flattening the ORDER column (from JSON) for simplicity

In [72]:
# Create a list to collect rows
rows = []

# Loop through each row of your order_data DataFrame
for idx, row in order_data.iterrows():
    try:
        order_json = json.loads(row['ORDERS'])  # Load JSON
        item_details = order_json['orders'][0]['item_details']  # Navigate to item list

        # For each item in the order, extract details
        for item in item_details:
            rows.append({
                'ORDER_ID': row['ORDER_ID'],
                'CUSTOMER_ID': row['CUSTOMER_ID'],
                'STORE_NUMBER': row['STORE_NUMBER'],
                'ORDER_CREATED_DATE': row['ORDER_CREATED_DATE'],
                'ORDER_CHANNEL_NAME': row['ORDER_CHANNEL_NAME'],
                'ORDER_SUBCHANNEL_NAME': row['ORDER_SUBCHANNEL_NAME'],
                'ORDER_OCCASION_NAME': row['ORDER_OCCASION_NAME'],
                'item_name': item.get('item_name'),
                'item_price': item.get('item_price'),
                'item_quantity': item.get('item_quantity')
            })
    except Exception as e:
        continue  # Skip problematic rows silently

# Convert to DataFrame
flat_orders = pd.DataFrame(rows)

In [73]:
flat_orders

,ORDER_ID,CUSTOMER_ID,STORE_NUMBER,ORDER_CREATED_DATE,ORDER_CHANNEL_NAME,ORDER_SUBCHANNEL_NAME,ORDER_OCCASION_NAME,item_name,item_price,item_quantity
0,7247194287,362204699,2156,2024-07-24,Digital,WWT,ToGo,Order Memo Not Paid,0.00,1
1,7247194287,362204699,2156,2024-07-24,Digital,WWT,ToGo,10 pc Grilled Wings Combo,15.29,1
2,7247194287,362204699,2156,2024-07-24,Digital,WWT,ToGo,8 pc Grilled Wings Combo,13.29,1
3,7247194287,362204699,2156,2024-07-24,Digital,WWT,ToGo,8 pc Spicy Wings Combo,10.99,1
4,791214421,269612955,1419,2025-02-15,Digital,WWT,ToGo,Ranch Dip - Regular,1.59,1
...,...,...,...,...,...,...,...,...,...,...
5657822,8865782702,125634365,1419,2024-11-30,Digital,WWT,ToGo,Order Memo Paid,0.00,2
5657823,8865782702,125634365,1419,2024-11-30,Digital,WWT,ToGo,Chicken Sub Combo,17.98,2
5657824,8030111101,125634365,4595,2024-12-16,Digital,WWT,ToGo,Ranch Dip - Regular,1.59,1
5657825,8030111101,125634365,4595,2024-12-16,Digital,WWT,ToGo,10 pc Spicy Wings,7.00,1


In [77]:
flat_orders.groupby(by='CUSTOMER_ID')['ORDER_ID'].count().sort_values(ascending=False).head(10)

CUSTOMER_ID
522408835    7574
300120733     732
575231786     709
313009408     619
29312308      503
573222710     500
807861401     472
927955293     469
963730416     457
413517928     447
Name: ORDER_ID, dtype: int64

In [76]:
flat_orders.groupby(by='CUSTOMER_ID')['ORDER_ID'].nunique().sort_values(ascending=False).head(10)

CUSTOMER_ID
522408835    2802
575231786     163
300120733     154
807861401     108
377803146     107
593414163      97
305682849      96
22394267       93
757801678      93
418779566      90
Name: ORDER_ID, dtype: int64

In [78]:
order_data

,CUSTOMER_ID,STORE_NUMBER,ORDER_CREATED_DATE,ORDER_ID,ORDERS,ORDER_CHANNEL_NAME,ORDER_SUBCHANNEL_NAME,ORDER_OCCASION_NAME
0,362204699,2156,2024-07-24,7247194287,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo
1,269612955,1419,2025-02-15,791214421,"{""orders"": [{""item_details"": [{""item_name"": ""R...",Digital,WWT,ToGo
2,585330633,2249,2025-02-15,7575285208,"{""orders"": [{""item_details"": [{""item_name"": ""2...",Digital,WWT,ToGo
3,950661333,2513,2024-03-29,4253875716,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo
4,434985772,1754,2024-04-08,7150407872,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo
...,...,...,...,...,...,...,...,...
1414405,125634365,4595,2025-02-02,762304305,"{""orders"": [{""item_details"": [{""item_name"": ""1...",Digital,WWT,ToGo
1414406,125634365,1419,2025-01-22,9371418183,"{""orders"": [{""item_details"": [{""item_name"": ""C...",Digital,WWT,ToGo
1414407,125634365,4595,2025-02-13,8082200700,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo
1414408,125634365,1419,2024-11-30,8865782702,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo


In [79]:
def count_items(order_json):
    try:
        data = json.loads(order_json)
        return len(data['orders'][0]['item_details'])
    except:
        return 0

order_data['num_items'] = order_data['ORDERS'].apply(count_items)

In [80]:
order_data

,CUSTOMER_ID,STORE_NUMBER,ORDER_CREATED_DATE,ORDER_ID,ORDERS,ORDER_CHANNEL_NAME,ORDER_SUBCHANNEL_NAME,ORDER_OCCASION_NAME,num_items
0,362204699,2156,2024-07-24,7247194287,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo,4
1,269612955,1419,2025-02-15,791214421,"{""orders"": [{""item_details"": [{""item_name"": ""R...",Digital,WWT,ToGo,4
2,585330633,2249,2025-02-15,7575285208,"{""orders"": [{""item_details"": [{""item_name"": ""2...",Digital,WWT,ToGo,2
3,950661333,2513,2024-03-29,4253875716,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo,7
4,434985772,1754,2024-04-08,7150407872,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo,7
...,...,...,...,...,...,...,...,...,...
1414405,125634365,4595,2025-02-02,762304305,"{""orders"": [{""item_details"": [{""item_name"": ""1...",Digital,WWT,ToGo,2
1414406,125634365,1419,2025-01-22,9371418183,"{""orders"": [{""item_details"": [{""item_name"": ""C...",Digital,WWT,ToGo,4
1414407,125634365,4595,2025-02-13,8082200700,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo,2
1414408,125634365,1419,2024-11-30,8865782702,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo,2


In [81]:
order_data.groupby(by='CUSTOMER_ID')['num_items'].sum().sort_values(ascending=False).head(10)

CUSTOMER_ID
522408835    7574
300120733     732
575231786     709
313009408     619
29312308      503
573222710     500
807861401     472
927955293     469
963730416     457
413517928     447
Name: num_items, dtype: int64

In [82]:
order_data['num_items'].sum()

5657827

In [83]:
flat_orders.shape

(5657827, 10)

In [84]:
flat_orders

,ORDER_ID,CUSTOMER_ID,STORE_NUMBER,ORDER_CREATED_DATE,ORDER_CHANNEL_NAME,ORDER_SUBCHANNEL_NAME,ORDER_OCCASION_NAME,item_name,item_price,item_quantity
0,7247194287,362204699,2156,2024-07-24,Digital,WWT,ToGo,Order Memo Not Paid,0.00,1
1,7247194287,362204699,2156,2024-07-24,Digital,WWT,ToGo,10 pc Grilled Wings Combo,15.29,1
2,7247194287,362204699,2156,2024-07-24,Digital,WWT,ToGo,8 pc Grilled Wings Combo,13.29,1
3,7247194287,362204699,2156,2024-07-24,Digital,WWT,ToGo,8 pc Spicy Wings Combo,10.99,1
4,791214421,269612955,1419,2025-02-15,Digital,WWT,ToGo,Ranch Dip - Regular,1.59,1
...,...,...,...,...,...,...,...,...,...,...
5657822,8865782702,125634365,1419,2024-11-30,Digital,WWT,ToGo,Order Memo Paid,0.00,2
5657823,8865782702,125634365,1419,2024-11-30,Digital,WWT,ToGo,Chicken Sub Combo,17.98,2
5657824,8030111101,125634365,4595,2024-12-16,Digital,WWT,ToGo,Ranch Dip - Regular,1.59,1
5657825,8030111101,125634365,4595,2024-12-16,Digital,WWT,ToGo,10 pc Spicy Wings,7.00,1


In [85]:
flat_orders.isna().sum()

ORDER_ID                 0
CUSTOMER_ID              0
STORE_NUMBER             0
ORDER_CREATED_DATE       0
ORDER_CHANNEL_NAME       0
ORDER_SUBCHANNEL_NAME    0
ORDER_OCCASION_NAME      0
item_name                0
item_price               0
item_quantity            0
dtype: int64

In [86]:
flat_orders.duplicated().any()

True

In [87]:
flat_orders.duplicated().sum()

540

In [88]:
# Show all duplicated rows
flat_orders[flat_orders.duplicated(keep=False)]

,ORDER_ID,CUSTOMER_ID,STORE_NUMBER,ORDER_CREATED_DATE,ORDER_CHANNEL_NAME,ORDER_SUBCHANNEL_NAME,ORDER_OCCASION_NAME,item_name,item_price,item_quantity
4409,3003957630,581545363,1765,2024-10-05,Digital,WWT,ToGo,8 pc Grilled Wings Combo,14.59,1
4413,3003957630,581545363,1765,2024-10-05,Digital,WWT,ToGo,8 pc Grilled Wings Combo,14.59,1
45410,976315004,403755278,2156,2024-02-26,Digital,WWT,ToGo,Ranch Dip - Regular,1.39,1
45412,976315004,403755278,2156,2024-02-26,Digital,WWT,ToGo,Ranch Dip - Regular,1.39,1
57719,1072774197,387761201,4094,2024-02-03,Digital,WWT,ToGo,20pc Spicy Feast Deal,16.99,1
...,...,...,...,...,...,...,...,...,...,...
5639080,9887154702,433921578,400,2024-04-20,Digital,WWT,ToGo,8pc Grilled Box,13.99,1
5645533,6561604583,261008456,400,2025-01-28,Digital,WWT,ToGo,Ranch Dip - Regular,1.59,1
5645534,6561604583,261008456,400,2025-01-28,Digital,WWT,ToGo,Ranch Dip - Regular,1.59,1
5646964,5152421458,315756008,4094,2024-03-22,Digital,WWT,ToGo,10 pc Spicy Wings Combo,15.79,1


### Dropping the Duplicates to avoid Data Redundancy

In [89]:
flat_orders_no_duplicates = flat_orders.drop_duplicates()

flat_orders_no_duplicates

,ORDER_ID,CUSTOMER_ID,STORE_NUMBER,ORDER_CREATED_DATE,ORDER_CHANNEL_NAME,ORDER_SUBCHANNEL_NAME,ORDER_OCCASION_NAME,item_name,item_price,item_quantity
0,7247194287,362204699,2156,2024-07-24,Digital,WWT,ToGo,Order Memo Not Paid,0.00,1
1,7247194287,362204699,2156,2024-07-24,Digital,WWT,ToGo,10 pc Grilled Wings Combo,15.29,1
2,7247194287,362204699,2156,2024-07-24,Digital,WWT,ToGo,8 pc Grilled Wings Combo,13.29,1
3,7247194287,362204699,2156,2024-07-24,Digital,WWT,ToGo,8 pc Spicy Wings Combo,10.99,1
4,791214421,269612955,1419,2025-02-15,Digital,WWT,ToGo,Ranch Dip - Regular,1.59,1
...,...,...,...,...,...,...,...,...,...,...
5657822,8865782702,125634365,1419,2024-11-30,Digital,WWT,ToGo,Order Memo Paid,0.00,2
5657823,8865782702,125634365,1419,2024-11-30,Digital,WWT,ToGo,Chicken Sub Combo,17.98,2
5657824,8030111101,125634365,4595,2024-12-16,Digital,WWT,ToGo,Ranch Dip - Regular,1.59,1
5657825,8030111101,125634365,4595,2024-12-16,Digital,WWT,ToGo,10 pc Spicy Wings,7.00,1


In [90]:
flat_orders.shape, flat_orders_no_duplicates.shape

((5657827, 10), (5657287, 10))

# Merging

In [96]:
y = flat_orders_no_duplicates.merge(customer_data, on='CUSTOMER_ID', how='left').merge(
    store_data, on='STORE_NUMBER', how='left'
)

y

,ORDER_ID,CUSTOMER_ID,STORE_NUMBER,ORDER_CREATED_DATE,ORDER_CHANNEL_NAME,ORDER_SUBCHANNEL_NAME,ORDER_OCCASION_NAME,item_name,item_price,item_quantity,CUSTOMER_TYPE,CITY,STATE,POSTAL_CODE,city_filled,state_filled,state_abbrev
0,7247194287,362204699,2156,2024-07-24,Digital,WWT,ToGo,Order Memo Not Paid,0.00,1,Registered,GRAPEVINE,TX,76051,Grapevine,Texas,TX
1,7247194287,362204699,2156,2024-07-24,Digital,WWT,ToGo,10 pc Grilled Wings Combo,15.29,1,Registered,GRAPEVINE,TX,76051,Grapevine,Texas,TX
2,7247194287,362204699,2156,2024-07-24,Digital,WWT,ToGo,8 pc Grilled Wings Combo,13.29,1,Registered,GRAPEVINE,TX,76051,Grapevine,Texas,TX
3,7247194287,362204699,2156,2024-07-24,Digital,WWT,ToGo,8 pc Spicy Wings Combo,10.99,1,Registered,GRAPEVINE,TX,76051,Grapevine,Texas,TX
4,791214421,269612955,1419,2025-02-15,Digital,WWT,ToGo,Ranch Dip - Regular,1.59,1,Registered,HUNTERSVILLE,NC,28078,Huntersville,North Carolina,NC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5657282,8865782702,125634365,1419,2024-11-30,Digital,WWT,ToGo,Order Memo Paid,0.00,2,Guest,HUNTERSVILLE,NC,28078,Huntersville,North Carolina,NC
5657283,8865782702,125634365,1419,2024-11-30,Digital,WWT,ToGo,Chicken Sub Combo,17.98,2,Guest,HUNTERSVILLE,NC,28078,Huntersville,North Carolina,NC
5657284,8030111101,125634365,4595,2024-12-16,Digital,WWT,ToGo,Ranch Dip - Regular,1.59,1,Guest,CHARLOTTE,NC,28208,Charlotte,North Carolina,NC
5657285,8030111101,125634365,4595,2024-12-16,Digital,WWT,ToGo,10 pc Spicy Wings,7.00,1,Guest,CHARLOTTE,NC,28208,Charlotte,North Carolina,NC


In [97]:
y.isna().sum()

ORDER_ID                 0
CUSTOMER_ID              0
STORE_NUMBER             0
ORDER_CREATED_DATE       0
ORDER_CHANNEL_NAME       0
ORDER_SUBCHANNEL_NAME    0
ORDER_OCCASION_NAME      0
item_name                0
item_price               0
item_quantity            0
CUSTOMER_TYPE            0
CITY                     0
STATE                    0
POSTAL_CODE              0
city_filled              0
state_filled             0
state_abbrev             0
dtype: int64

In [98]:
y.duplicated().any()

False

In [99]:
y['item_name'].nunique()

145

In [100]:
y['item_name'].unique()

array(['Order Memo Not Paid', '10 pc Grilled Wings Combo',
       '8 pc Grilled Wings Combo', '8 pc Spicy Wings Combo',
       'Ranch Dip - Regular', '50 pc Grilled Wings',
       'Regular Buffalo Fries', 'Order Memo Paid',
       '20pc Spicy Feast Deal', 'Order Memo Item', '20 pc Grilled Wings',
       'Order Memo ASAP', 'Order Blankline 2', 'Order Blankline 1',
       '6 pc Grilled Wings Combo', '10 pc Grilled Wings',
       'Fried Corn - Regular', 'Ranch Dip - Large', 'Veggie Sticks Spicy',
       'Flavor Platter', 'Large Fruit Punch',
       'Honey Mustard Dip - Regular', 'Voodoo Fries - Regular',
       '10 pc Spicy Wings', '2 pc Crispy Strips', 'Add 5 Grilled Wings',
       '24 pc Family Grilled Wings', '6 pc Spicy Wings Combo',
       'Chicken Sub Combo', '15 pc Spicy Wings',
       '10 pc Spicy Wings Combo', 'Add 5 Spicy Wings',
       'Blue Cheese Dip - Regular', 'Chicken Sub',
       '30 pc Family Grilled Wings', 'Order Memo Future',
       '10 pc Mixed Wings Combo', 'Dipping

In [101]:
y.to_csv('final_merged_data.csv', index=False)